## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The example uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "ml-in-use" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-54-145-40-85.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-54-145-40-85.compute-1.amazonaws.com:5000'


In [3]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://mlflow-artifact-remote-1/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifact-remote-1/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 's3://mlflow-artifact-remote-1/1/444a6124bf8245258b9c21c52574f234/artifacts'


In [6]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://mlflow-artifact-remote-1/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifact-remote-1/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [8]:
client.list_registered_models()

[]

In [9]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2022/06/01 19:35:39 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1654108539174, current_stage='None', description='', last_updated_timestamp=1654108539174, name='iris-classifier', run_id='444a6124bf8245258b9c21c52574f234', run_link='', source='s3://mlflow-artifact-remote-1/1/444a6124bf8245258b9c21c52574f234/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [10]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1654108538593, description='', last_updated_timestamp=1654108539174, latest_versions=[<ModelVersion: creation_timestamp=1654108539174, current_stage='None', description='', last_updated_timestamp=1654108539174, name='iris-classifier', run_id='444a6124bf8245258b9c21c52574f234', run_link='', source='s3://mlflow-artifact-remote-1/1/444a6124bf8245258b9c21c52574f234/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]